In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('animelists_cleaned.csv/animelists_cleaned.csv')

In [3]:
df.head()

,username,anime_id,my_watched_episodes,my_start_date,my_finish_date,my_score,my_status,my_rewatching,my_rewatching_ep,my_last_updated,my_tags
0,karthiga,21,586,0000-00-00,0000-00-00,9,1,NaN,0,2013-03-03 10:52:53,NaN
1,karthiga,59,26,0000-00-00,0000-00-00,7,2,NaN,0,2013-03-10 13:54:51,NaN
2,karthiga,74,26,0000-00-00,0000-00-00,7,2,NaN,0,2013-04-27 16:43:35,NaN
3,karthiga,120,26,0000-00-00,0000-00-00,7,2,NaN,0,2013-03-03 10:53:57,NaN
4,karthiga,178,26,0000-00-00,0000-00-00,7,2,0.0,0,2013-03-27 15:59:13,NaN


In [4]:
df.shape

(31284030, 11)

In [6]:
import operator

In [112]:
def calculate_order_greedy(Item_list, rating, user_id):
    pie = {}
    rank = {}
    for i in Item_list:
        pie[i] = preference_func(i, i+1, user_id, rating) - preference_func(i+1, i, user_id, rating)
    while (len(Item_list) > 0):
        t = max(pie.items(), key = operator.itemgetter(1))[0]
        rank[t] = len(Item_list)
        Item_list.remove(t)
        for k in Item_list:
            pie[k] = pie[k] + preference_func(t,k, user_id, rating) - preference_func(k,t, user_id, rating)
    return rank
        
        

In [14]:
rating = pd.read_csv('rating.csv/rating.csv')

In [122]:
mask =(rating['user_id'].value_counts() >= 10) & (rating['user_id'].value_counts() <= 15)


In [123]:
users = mask[mask == True].index

In [124]:
rating_2 = rating[rating['user_id'].isin(users)]

In [125]:
rating_2.shape

(63270, 3)

In [126]:
rating_2['anime_id'].nunique()

3275

In [78]:
rating.shape

(6337241, 3)

In [16]:
rating = rating[rating['rating'] >0]


In [48]:
rating[rating['user_id'] == 3]

,user_id,anime_id,rating
156,3,20,8
157,3,154,6
158,3,170,9
159,3,199,10
160,3,225,9
161,3,341,6
162,3,430,7
163,3,527,7
164,3,552,7
165,3,813,10


In [99]:
def indicator_f(x):
    if(x <0):
        r = 1
    else:
        r = 0
    return r

def Kendall_CC(u_id, v_id, df):
    S = 0
    
    user_anm_rating = pd.merge(df[df['user_id'] == u_id], df[df['user_id'] == v_id], on='anime_id', how='inner', \
                               suffixes={'_U', '_V'})
    if(len(user_anm_rating)>10):
        val = 0
        for a in range(len(user_anm_rating)-1):
            e = (user_anm_rating.loc[a,'rating_U'] - user_anm_rating.loc[a+1,'rating_U'])*(user_anm_rating.loc[a,'rating_V'] - user_anm_rating.loc[a+1,'rating_V'])
            val = val + indicator_f(e)
            #print(val)
        S = 1-4*(val/len(user_anm_rating)*(len(user_anm_rating)-1))
    return S
        
        
    

In [32]:
user_anm_rating= pd.merge(rating[rating['user_id'] == 3], rating[rating['user_id'] == 5], on='anime_id', how='inner', suffixes={'_U', '_V'}) 

In [144]:
def preference_func(A, B, user_id, rating):
    neighbs = Neighb(user_id, rating) ## return dictionary
    num = 0.0
    denom = 0.0
    for key, val in neighbs.items():
        prod = rating.loc[((rating['user_id'] == key)&(rating['anime_id'] == A)), 'rating'] - rating.loc[((rating['user_id'] == key)&(rating['anime_id'] == B)), 'rating']
        num = num + prod*val
        denom = denom+val
    return num/denom

In [50]:
###preference_func(1535,20,3 )
import numpy as np

In [140]:
def Neighb(user_id, rating, cutoff = 10):
    all_users = list((rating['user_id']).unique())#.remove(user_id)
    all_users.remove(user_id)
    neighb_score = {}
    for u in all_users:
        #print(u)
        neighb_score[u] = Kendall_CC(user_id, u, rating)
    top_10 = sorted(neighb_score.items(), key=operator.itemgetter(1), reverse=True)[:cutoff]
    return dict(top_10)
        
    

In [105]:
##Neighb(3, rating_2)

In [62]:
user_anm_rating= pd.merge(rating[rating['user_id'] == 3], rating[rating['user_id'] == 9], on='anime_id', how='inner', suffixes={'_U', '_V'}) 

In [142]:
d = {'a':10, 'b':12, 'c':1, 'd':0.77}
d = dict(sorted(d.items(), key=operator.itemgetter(1), reverse=True)[:3])

In [127]:
all_items = rating_2['anime_id'].unique()

In [146]:
len(all_items)

3275

In [145]:
all_ranks = calculate_order_greedy(all_items, rating_2, 28)

KeyboardInterrupt: 

In [147]:
## Stopped due to long time